In [6]:
import cv2
import os
import numpy as np

def segment():
    print("Segmenting")
    # set path
    path=os.path.join(os.getcwd(),'good/')
    
    # read input image
    inputfile = cv2.imread(path+'MaI12_Page4.jpg.png')

    # shadow removal
    rgb_planes=cv2.split(inputfile)
    result_planes=[]
    result_norm_planes=[]
    kernel = np.ones((7,7),np.uint8)
    for plane in rgb_planes:
        dilated_img=cv2.dilate(plane,kernel)
        bg_img=cv2.medianBlur(dilated_img,21) # scratch text out
        diff_img=255-cv2.absdiff(plane,bg_img)
        norm_img=cv2.normalize(diff_img,diff_img,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8UC1)
        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)
    result=cv2.merge(result_norm_planes)
    cv2.imwrite(path+'shadow_removed.jpg',result)

    # line segmentation
    gray = cv2.imread(path+'shadow_removed.jpg',0)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    ret,line_binary = cv2.threshold(blur,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    line_binary=255-line_binary
    line_image=line_binary
    
    # find line contours
    line_kernel = np.ones((3,line_image.shape[1]//8), np.uint8)
    line_closed=cv2.morphologyEx(line_binary,cv2.MORPH_CLOSE,line_kernel)
    line_ctrs, line_hier = cv2.findContours(line_closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # sort line contours
    line_sorted_ctrs = sorted(line_ctrs, key=lambda line_ctr: cv2.boundingRect(line_ctr)[1])

    # loop over each line
    for line_i, line_ctr in enumerate(line_sorted_ctrs):
        line_x, line_y, line_w, line_h = cv2.boundingRect(line_ctr)

        # Getting ROI
        line_roi = line_image[line_y:line_y+line_h, line_x:line_x+line_w]
        line_roi_binary=line_binary[line_y:line_y+line_h,line_x:line_x+line_w]
        line_mask=np.zeros_like(line_binary)
        cv2.drawContours(line_mask,[line_ctr],0, (255,255,255), -1)
        line_roi_mask=line_mask[line_y:line_y+line_h,line_x:line_x+line_w]
        line_roi_binary=cv2.bitwise_and(line_roi_binary,line_roi_mask)
        cv2.imwrite(path+str(line_i+1)+'_line_roi_binary.jpg',line_roi_binary)

        # character segmentation
        char_kernel = np.ones((line_roi.shape[0]//3, 3), np.uint8)
        char_closed=cv2.morphologyEx(line_roi_binary,cv2.MORPH_CLOSE,char_kernel)
        char_ctrs, char_hier = cv2.findContours(char_closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # loop over each character
        for char_i, char_ctr in enumerate(char_ctrs):
            char_x, char_y, char_w, char_h = cv2.boundingRect(char_ctr)
            
            
segment()

Segmenting
